In [1]:
import os
import glob
import shutil
import numpy as np
import pandas as pd
import random
from PIL import Image


In [2]:
from config import esri_retinanet_config as config
from bs4 import BeautifulSoup
from imutils import paths
import argparse
import random
import os


In [3]:
#os.chdir("./retinanet/")
files = glob.glob('./dataset/images/*')
for f in files:
    os.remove(f)
 
files = glob.glob('./dataset/annotations/*')
for f in files:
    os.remove(f)
    
case = "rgb"
lbls = "taxonID"
sites = ["OSBS"]

In [4]:
#reset dataset folder and include only 

In [5]:
ls_f = (glob.glob('../'+case+'/*.tif'))
for f in ls_f:
    shutil.copy(f, './dataset/images/')
    
ls_f = (glob.glob('../csv_labels/*'+case+'.csv'))
for f in ls_f:
    shutil.copy(f, './dataset/annotations/')

In [6]:
# initialize the base path for the logos dataset
BASE_PATH = "dataset"

# build the path to the annotations and input images
annot_path = os.path.sep.join([BASE_PATH, 'annotations'])
images_path = os.path.sep.join([BASE_PATH, 'images'])

# define the training/testing split
train_test_split = 0.8

#  build the path to the output training and test .csv files
train_csv = os.path.sep.join([BASE_PATH, 'train.csv'])
test_csv = os.path.sep.join([BASE_PATH, 'test.csv'])

# build the path to the output classes CSV files
classes_csv = os.path.sep.join([BASE_PATH, 'classes.csv'])

# build the path to the output predictions dir
OUTPUT_DIR = os.path.sep.join([BASE_PATH, 'predictions'])

In [7]:
# grab all image paths then construct the training and testing split
imagePaths = list(paths.list_files(images_path))
random.seed( 30 )
random.shuffle(imagePaths)
i = int(len(imagePaths) * train_test_split)
trainImagePaths = imagePaths[:i]
testImagePaths = imagePaths[i:]

# create the list of datasets to build
dataset = [ ("train", trainImagePaths, train_csv),
            ("test", testImagePaths, test_csv)]

# initialize the set of classes we have
CLASSES = set()

In [8]:
veg_structure = pd.read_csv("vegetation_structure.csv")
veg_structure = veg_structure[["individualID", "siteID", lbls]]
#is_sites = veg_structure.siteID.isin(list(sites))
veg_structure = veg_structure[veg_structure.siteID.isin(list(sites))]
list_labels = veg_structure[lbls].unique()
lb_code = range(len(list_labels)+1)
labelnet = {}
for i in range(len(list_labels)):
    labelnet[list_labels[i]] = lb_code[i+1]


/Users/sergiomarconi/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
labelnet

{'QULA2': 1,
 'PIPA2': 2,
 'QUGE2': 3,
 'DIOSP': 4,
 'PINUS': 5,
 'QUNI': 6,
 'ACRU': 7,
 'ILCA': 8,
 'MOCE2': 9,
 'PEPA37': 10,
 'GOLA': 11,
 'LYLU3': 12,
 'PIEL': 13,
 'NYBI': 14,
 'QUHE2': 15,
 'CEOC': 16,
 'ILMY': 17,
 'PITA': 18,
 'QULA3': 19,
 'QUIN': 20,
 'QUVI': 21,
 'VIRO3': 22,
 'ASIN12': 23,
 'QUMA13': 24,
 'VAAR': 25,
 'SAET': 26}

In [10]:
for (dType, imagePaths, outputCSV) in dataset:
    # load the contents
    print ("[INFO] creating '{}' set...".format(dType))
    print ("[INFO] {} total images in '{}' set".format(len(imagePaths), dType))

    # open the output CSV file
    #csv = open(outputCSV, "w")

    # loop over the image paths
    labels = pd.DataFrame()
    for imagePath in imagePaths:
        fl = imagePath.split("/")[2][:-4]
        tmp = pd.read_csv('./dataset/annotations/'+fl+'.csv')
        labels = labels.append(tmp, sort=False)
        
    labels = labels.rename(columns={'label': 'individualID'})
    labels = labels.join(veg_structure.set_index('individualID'), on='individualID')
    labels=labels.replace({lbls: labelnet})
    labels = labels.rename(columns={lbls: 'label'})


    labels.to_csv("./"+outputCSV)
    # update the set of unique class labels
    #CLASSES.add(label)

    
        



[INFO] creating 'train' set...
[INFO] 32 total images in 'train' set
[INFO] creating 'test' set...
[INFO] 9 total images in 'test' set


In [52]:
#pd.DataFrame.from_dict(labelnet)
new_dict = {k: [v] for k, v in labelnet.items()}
df = pd.DataFrame(new_dict)
df = df.T
df.to_csv("./species_dictionary.csv")

In [53]:
df['index'] = list(range(len(df)))
print(df)
df.to_csv("./"+classes_csv, header=False, index = False)

         0  index
QULA2    1      0
PIPA2    2      1
QUGE2    3      2
DIOSP    4      3
PINUS    5      4
QUNI     6      5
ACRU     7      6
ILCA     8      7
MOCE2    9      8
PEPA37  10      9
GOLA    11     10
LYLU3   12     11
PIEL    13     12
NYBI    14     13
QUHE2   15     14
CEOC    16     15
ILMY    17     16
PITA    18     17
QULA3   19     18
QUIN    20     19
QUVI    21     20
VIRO3   22     21
ASIN12  23     22
QUMA13  24     23
VAAR    25     24
SAET    26     25
